In [3]:
import sys
import os

# Assumes the current working directory is the 'usage' folder in this repo. 
parent_dir = os.path.abspath('..')
sys.path.append(parent_dir)

# Internal imports. 
from blue_rov import Params as blue # blue_rov.py in main folder. 
from diffUV import dyn_body, kin, dyned_eul, dyned_quat # names defined in __init__.py. 
from diffUV.utils.symbols import *
from diffUV.utils.quaternion_ops import euler2q, q2euler
from diffUV import simulator # names defined in __init__.py. 
from diffUV import control # names defined in __init__.py. 

# External imports. 
from casadi import *
import numpy as np
import os
from scipy.spatial.transform import Rotation as R

import matplotlib.pyplot as plt

In [4]:
# body representaion
uv_body = dyn_body()

# ned representaion
uv_ned = dyned_eul()

# quaternion representation
uv_quat = dyned_quat()

# kinematics representation
Kinematics = kin()

In [5]:
import math

def euler_from_quaternion(q):
        w, x, y, z = q
        """
        Convert a quaternion into euler angles (roll, pitch, yaw)
        roll is rotation around x in radians (counterclockwise)
        pitch is rotation around y in radians (counterclockwise)
        yaw is rotation around z in radians (counterclockwise)
        """
        t0 = +2.0 * (w * x + y * z)
        t1 = +1.0 - 2.0 * (x * x + y * y)
        roll_x = math.atan2(t0, t1)

        t2 = +2.0 * (w * y - z * x)
        t2 = +1.0 if t2 > +1.0 else t2
        t2 = -1.0 if t2 < -1.0 else t2
        pitch_y = math.asin(t2)

        t3 = +2.0 * (w * z + x * y)
        t4 = +1.0 - 2.0 * (y * y + z * z)
        yaw_z = math.atan2(t3, t4)

        return np.array([roll_x, pitch_y, yaw_z]) # in radians